In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/hubmap-kidney-segmentation/train.csv')

In [ ]:
len(df)

In [ ]:
import cv2
import tifffile
import matplotlib.pyplot as plt

In [ ]:
input_dir = '../input/hubmap-kidney-segmentation/train'

In [ ]:
def rle2mask(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
os.makedirs('../working/split/images')
os.makedirs('../working/split/masks')
split_size = 1024

In [ ]:
for im_name in os.listdir(input_dir):
    if '.tiff' in im_name:
        print(im_name)
        image_path = os.path.join(input_dir, im_name)
        im = tifffile.imread(os.path.join(input_dir, im_name))
        mask_rle = df[df["id"] == im_name[:-5]]["encoding"].values[0]
        mask = rle2mask(df[df["id"] == im_name[:-5]]["encoding"].values[0], (im.shape[1], im.shape[0]))*255
        for r in range(0, im.shape[0], split_size):
            for c in range(0, im.shape[1], 1024):
                im_tile = im[r: r + split_size, c: c + split_size, :]
                mask_tile = mask[r: r + split_size, c: c + split_size]
            # here I filter images with 0-mask and white borders around.
                if (np.sum(mask_tile)==0):
                    if ((2 * split_size <= r <= (im.shape[0] - 2 * split_size)) and \
                        (2 * split_size <= c <= (im.shape[1] - 2 * split_size))):
                        tifffile.imwrite(f"split/images/img{r}_{c}.png", im_tile)
                        tifffile.imwrite(f"split/masks/img{r}_{c}.png", mask_tile)
                else:
                    tifffile.imwrite(f"split/images/img{r}_{c}.png", im_tile)
                    tifffile.imwrite(f"split/masks/img{r}_{c}.png", mask_tile)

In [ ]:
!ls split/images/